Types of concept drift:
- Sudden Drift
- Gradual Drift
- Recurrent Drift
- Real vs Virtual (see second article)
- Blip = Outlier (see second article)
- Incremental (see second article)


https://neptune.ai/blog/concept-drift-best-practices

https://www.sciencedirect.com/science/article/pii/S1319157821003062

weight the input data (new data weighted more heavily than past data)

https://machinelearningmastery.com/gentle-introduction-concept-drift-machine-learning/

In [ ]:
#reading chunk by chunk

import pandas as pd
import glob
from collections import deque

all_files = glob.glob(r'/nfs/home/ikq2724/data/*.csv') # File Path to read csv
li = []

window_len = 100
buffer = deque()
for file in all_files[:1]:
    window_start = 0
    dfs = pd.read_csv(all_files[0], iterator = True, chunksize = window_len)
    for idx, df in enumerate(dfs):
        ## Use this df chunk
        buffer.append(df)

1
- tuning multiflow parameters
- how quickly it detecs drift
- windowsize

- detecting concept drift
- this is the only implementation for concept drift

2
- look into whether functions retrain model once it detects drift
- valid use case to retrain model is drift is detected


3
- introduce concept drift to real data

- randomly change (80/20 or 50/50), change the actual labels, for a section of the data
- change for all data in that section; change for a % of data in that section
- ramp up change/ramp down change

- what labels we flip should be randomized; 
- for each datapoint, generate random number (set a threshold), if random number > threshold flip label


4
- apply on real data



- papers more useful

In [26]:
import numpy as np
from skmultiflow.drift_detection import DDM
from skmultiflow.drift_detection import ADWIN
from skmultiflow.drift_detection import EDDM
from skmultiflow.drift_detection import PageHinkley
import matplotlib.pyplot as plt
import pandas as pd

In [27]:
# ddm = DDM()
# data_stream = np.random.randint(2,size=2000)
# #data_stream = np.zeros(2000)
# for i in range(999,1500):
#     data_stream[i] = 1
# for i in range(2000):
#     ddm.add_element(data_stream[i])
#     if ddm.detected_warning_zone():
#         print('Warning zone has been detected in data: ' + str(data_stream[i]) + ' - of index: ' + str(i))
#     if ddm.detected_change():
#         print('Change has been detected in data: ' + str(data_stream[i]) + ' - of index: ' + str(i))


# adwin = ADWIN()
# data_stream = np.random.randint(2,size=2000)
# #data_stream = np.zeros(2000)
# for i in range(999,1500):
#     data_stream[i] = 1
# for i in range(2000):
#     adwin.add_element(data_stream[i])
#     if adwin.detected_warning_zone():
#         print('Warning zone has been detected in data: ' + str(data_stream[i]) + ' - of index: ' + str(i))
#     if adwin.detected_change():
#         print('Change has been detected in data: ' + str(data_stream[i]) + ' - of index: ' + str(i))

In [48]:
## Creating the relevant dataset
num_samples = 10000
np.random.seed(0)
x_1 = np.random.uniform(low=0,high=1,size=num_samples)
np.random.seed(5)
x_2 = np.random.uniform(low=0,high=1,size=num_samples)

x_sin = np.sin(x_1)

df_sine_train = pd.DataFrame({"x1":x_1,"x2":x_2,"x_sin":x_sin})
df_sine_train["target"] = np.where(df_sine_train["x2"]>df_sine_train["x_sin"],1,0)
df_sine_train.drop("x_sin",axis=1,inplace=True)

df_sine_train["target"].value_counts()
#df_sine_train

1    5491
0    4509
Name: target, dtype: int64

In [49]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier

In [50]:
X = df_sine_train.drop("target",axis=1)
y = df_sine_train["target"]

model_tree = DecisionTreeClassifier()
model_tree.fit(X,y)

y_predict  = model_tree.predict(X)

confusion_matrix(y,y_predict)

array([[4509,    0],
       [   0, 5491]])

In [51]:
## Creating the relevant test dataset


df_sine_test = pd.DataFrame()

##
num_samples = 2000
np.random.seed(1)
x_1 = np.random.uniform(low=0,high=1,size=num_samples)
np.random.seed(2)
x_2 = np.random.uniform(low=0,high=1,size=num_samples)
x_sin = np.sin(x_1)

df_sine_test1 = pd.DataFrame({"x1":x_1,"x2":x_2,"x_sin":x_sin})
df_sine_test1["target"] = np.where(df_sine_test1["x2"]>df_sine_test1["x_sin"],1,0)
df_sine_test1.drop("x_sin",axis=1,inplace=True)

df_sine_test = pd.concat([df_sine_test,df_sine_test1])

##

num_samples = 2000
np.random.seed(10)
x_1 = np.random.uniform(low=0,high=1,size=num_samples)
np.random.seed(12)
x_2 = np.random.uniform(low=0,high=1,size=num_samples)
x_sin = np.sin(x_1)

df_sine_test1 = pd.DataFrame({"x1":x_1,"x2":x_2,"x_sin":x_sin})
df_sine_test1["target"] = np.where(df_sine_test1["x2"]<df_sine_test1["x_sin"],1,0)
df_sine_test1.drop("x_sin",axis=1,inplace=True)

df_sine_test = pd.concat([df_sine_test,df_sine_test1])

##

num_samples = 2000
np.random.seed(11)
x_1 = np.random.uniform(low=0,high=1,size=num_samples)
np.random.seed(15)
x_2 = np.random.uniform(low=0,high=1,size=num_samples)
x_sin = np.sin(x_1)

df_sine_test1 = pd.DataFrame({"x1":x_1,"x2":x_2,"x_sin":x_sin})
df_sine_test1["target"] = np.where(df_sine_test1["x2"]>df_sine_test1["x_sin"],1,0)
df_sine_test1.drop("x_sin",axis=1,inplace=True)

df_sine_test = pd.concat([df_sine_test,df_sine_test1])

df_sine_test.reset_index(drop=True,inplace=True)

X_test = df_sine_test.drop("target",axis=1)
y_test = df_sine_test["target"]

In [53]:
## Testing the drift detectors - drift occurs at 2000 and 4000 DDM

ddm = DDM()

for i in range(6000):
    df_curr = X_test.iloc[[i]]
    #print(df_curr)
    pred_df = model_tree.predict(df_curr)
    curr_pred = pred_df[0]
    
    if(curr_pred==y_test[i]):
        ddm.add_element(1)
    else:
        ddm.add_element(0)
        
    if ddm.detected_warning_zone():
        print('Warning zone has been detected in data: ' + str(i) )
    if ddm.detected_change():
        print('Change has been detected in data: ' + str(i) )
    

Warning zone has been detected in data: 4064
Warning zone has been detected in data: 4065
Warning zone has been detected in data: 4066
Warning zone has been detected in data: 4067
Warning zone has been detected in data: 4068
Warning zone has been detected in data: 4069
Warning zone has been detected in data: 4070
Warning zone has been detected in data: 4071
Warning zone has been detected in data: 4072
Warning zone has been detected in data: 4073
Warning zone has been detected in data: 4074
Warning zone has been detected in data: 4075
Warning zone has been detected in data: 4076
Warning zone has been detected in data: 4077
Warning zone has been detected in data: 4078
Warning zone has been detected in data: 4079
Warning zone has been detected in data: 4080
Warning zone has been detected in data: 4081
Warning zone has been detected in data: 4082
Warning zone has been detected in data: 4083
Warning zone has been detected in data: 4084
Warning zone has been detected in data: 4085
Warning zo

In [54]:
## Testing the drift detectors - drift occurs at 2000 and 4000 EDDM

eddm = EDDM()

for i in range(6000):
    df_curr = X_test.iloc[[i]]
    #print(df_curr)
    pred_df = model_tree.predict(df_curr)
    curr_pred = pred_df[0]
    
    if(curr_pred==y_test[i]):
        eddm.add_element(1)
    else:
        eddm.add_element(0)
        
    if eddm.detected_warning_zone():
        print('Warning zone has been detected in data: ' + str(i) )
    if eddm.detected_change():
        print('Change has been detected in data: ' + str(i) )
    

Warning zone has been detected in data: 4221
Warning zone has been detected in data: 4222
Warning zone has been detected in data: 4223
Warning zone has been detected in data: 4224
Warning zone has been detected in data: 4225
Warning zone has been detected in data: 4226
Warning zone has been detected in data: 4227
Warning zone has been detected in data: 4228
Warning zone has been detected in data: 4229
Warning zone has been detected in data: 4230
Warning zone has been detected in data: 4231
Warning zone has been detected in data: 4232
Warning zone has been detected in data: 4233
Warning zone has been detected in data: 4234
Warning zone has been detected in data: 4235
Warning zone has been detected in data: 4236
Warning zone has been detected in data: 4237
Warning zone has been detected in data: 4238
Warning zone has been detected in data: 4239
Warning zone has been detected in data: 4240
Warning zone has been detected in data: 4241
Warning zone has been detected in data: 4242
Warning zo

Warning zone has been detected in data: 4747
Warning zone has been detected in data: 4748
Warning zone has been detected in data: 4749
Warning zone has been detected in data: 4750
Warning zone has been detected in data: 4751
Warning zone has been detected in data: 4752
Warning zone has been detected in data: 4753
Warning zone has been detected in data: 4754
Warning zone has been detected in data: 4755
Warning zone has been detected in data: 4756
Warning zone has been detected in data: 4757
Warning zone has been detected in data: 4758
Warning zone has been detected in data: 4759
Warning zone has been detected in data: 4760
Warning zone has been detected in data: 4761
Warning zone has been detected in data: 4762
Warning zone has been detected in data: 4763
Warning zone has been detected in data: 4764
Warning zone has been detected in data: 4765
Warning zone has been detected in data: 4766
Warning zone has been detected in data: 4767
Warning zone has been detected in data: 4768
Warning zo

In [55]:
## Testing the drift detectors - drift occurs at 2000 and 4000

adwin = ADWIN()
test_mat = []

for i in range(6000):
    df_curr = X_test.iloc[[i]]
    #print(df_curr)
    pred_df = model_tree.predict(df_curr)
    curr_pred = pred_df[0]
    
    if(curr_pred==y_test[i]):
        adwin.add_element(1)
        test_mat.append(1)
    else:
        adwin.add_element(0)
        test_mat.append(0)
        
    if adwin.detected_warning_zone():
        #print("Warn")
        print('Warning zone has been detected in data: ' + str(i))
    if adwin.detected_change():
        #print("Change")
        print('Change has been detected in data: ' +  str(i))

#adwin.get_info()
#adwin.get_length_estimation()

Change has been detected in data: 2047
Change has been detected in data: 2079
Change has been detected in data: 2239
Change has been detected in data: 4031
Change has been detected in data: 4063
Change has been detected in data: 4095
Change has been detected in data: 4127
Change has been detected in data: 4159


In [56]:
## Testing the drift detectors - drift occurs at 2000 and 4000

phy = PageHinkley()
#test_mat = []

for i in range(6000):
    df_curr = X_test.iloc[[i]]
    #print(df_curr)
    pred_df = model_tree.predict(df_curr)
    curr_pred = pred_df[0]
    
    if(curr_pred==y_test[i]):
        phy.add_element(1)
    else:
        phy.add_element(0)
        
    if phy.detected_warning_zone():
        #print("Warn")
        print('Warning zone has been detected in data: ' + str(i))
    if phy.detected_change():
        #print("Change")
        print('Change has been detected in data: ' +  str(i))
    

Change has been detected in data: 4103
